# OSMnx Analysis
This analysis is useful to observe the overall structure and situation of the road network by relying on OSMnx. 
Specifically, dead-end nodes should be examined to determine where OSM updates are needed.

In [1]:
# Import packages
import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt



In [2]:
# Could be turned to False
# In that case "file_all_network" will be used.
download_data = True 

In [3]:
# The associated files
file_study_region = "study_region.geojson"
file_all_network = "saved_network.graphml" # the file saved, when download data is selected
file_nodes = "nodes.geojson"
file_edges = "edges_geojson"
file_dead_ends = "dead_ends.geojson"
epsg_roi = 32636 # WGS 84 / UTM zone 36N

In [4]:
if(download_data):
    # Load polygon and download network
    gdf = gpd.read_file("study_region.geojson")  # Replace with your shapefile or GeoJSON
    gdf_proj = gdf.to_crs(epsg=epsg_roi)  # replace EPSG with correct UTM for your area
    area_km2 = gdf_proj.geometry.area.sum() / 1e6
    print(f"Study area: {area_km2:.2f} km²") # 2.17 km^2
    polygon = gdf.geometry.union_all()
    
    G = ox.graph_from_polygon(polygon, network_type="all")
    ox.save_graphml(G, filepath=file_all_network)
    
    nodes, edges = ox.graph_to_gdfs(G)
    G_undirected = G.to_undirected()
    nodes.to_file("nodes.geojson", driver="GeoJSON")
    edges.to_file("edges.geojson", driver="GeoJSON")

Study area: 2.17 km²


In [5]:
G = ox.load_graphml(file_all_network)
G_undirected = G.to_undirected()

nodes, edges = ox.graph_to_gdfs(G)
num_nodes = len(nodes)
num_edges = len(edges)

# Overall statistics
print("# Nodes: ", num_nodes)
print("# of Edges: ", num_edges)

# Connected components
comps = list(nx.connected_components(G_undirected))
num_components = len(comps)
largest = max(len(c) for c in comps)
largest_pct = largest / num_nodes * 100
print("Connected component:",  largest_pct)


edges_proj = edges.to_crs(epsg=epsg_roi)  
lengths = edges_proj.geometry.length
total_length_m = lengths.sum()

if 'cycleway' in edges_proj.columns:
    bike_lane_len = edges_proj.loc[edges_proj['cycleway'].notna()].geometry.length.sum()
else:
    bike_lane_len = 0.0

bike_pct = (bike_lane_len / total_length_m) * 100

pedestrian_types = ['footway', 'pedestrian', 'path', 'living_street']
ped_len = edges_proj.loc[edges_proj['highway'].isin(pedestrian_types)].geometry.length.sum()

ped_pct = (ped_len / total_length_m) * 100

print(f"Total network length: {total_length_m/1000:.2f} km")
print(f"Bicycle path length: {bike_lane_len/1000:.2f} km ({bike_pct:.2f}%)")
print(f"Pedestrian path length: {ped_len/1000:.2f} km ({ped_pct:.2f}%)")

# Node degree analysis (dead-ends: degree = 1)
degrees = [d for _, d in G_undirected.degree()]
avg_degree = sum(degrees) / num_nodes
print(f"Avg node degree: {avg_degree:.2f}")



#Name-tag completeness
name_pct = edges['name'].notna().mean() * 100
print("Name-tag completeness (%): ", round(name_pct, 2))






# Nodes:  681
# of Edges:  1664
Connected component: 100.0
Total network length: 75.82 km
Bicycle path length: 0.00 km (0.00%)
Pedestrian path length: 29.21 km (38.52%)
Avg node degree: 2.67
Name-tag completeness (%):  10.76


# Dead end node analysis

In [6]:
# Export dead-ends
dead_ends = [n for n, d in G_undirected.degree() if d == 1]

# Create GeoDataFrame
geometries = [Point((G.nodes[n]['x'], G.nodes[n]['y'])) for n in dead_ends]
gdf = gpd.GeoDataFrame({'node': dead_ends}, geometry=geometries, crs='EPSG:4326')
print(f"Total dead-ends: {len(dead_ends)}  ({(len(dead_ends) / num_nodes)*100:.2f}%)")

# Save to GeoJSON
gdf.to_file(file_dead_ends, driver="GeoJSON")


Total dead-ends: 165  (24.23%)
